### Given a topic research the latest news

In [19]:
import sys
sys.path.append('..')

from dotenv import load_dotenv
load_dotenv()

from utils import utils , llms
import json

In [7]:
utils.set_prefix("../output/magnuscarlsen")
scenes = utils.get_scenes()
# Extract lines starting with "[Scene"
scene_lines = [s.description for s in scenes]
scene_lines

['The early life and exceptional talent of Magnus Carlsen.',
 "Carlsen's early exposure to puzzles, chess, and his incredible memory.",
 "Carlsen's rapid growth and early victories in the chess world.",
 "Carlsen's unique style and approach to chess.",
 "Carlsen's journey to becoming the World Chess Champion.",
 "An overview of Carlsen's career after 2006 and his personal life.",
 'Reminding viewers to like, subscribe, and highlighting the AI operation of the channel.',
 "Recapping Carlsen's achievements and expressing gratitude to the viewers."]

In [8]:

import torch
from diffusers import StableDiffusionPipeline

pipeline = StableDiffusionPipeline.from_pretrained("SG161222/Realistic_Vision_V2.0", torch_dtype=torch.float16)  

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [9]:
pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention()

In [10]:
pipeline.scheduler.compatibles

[diffusers.schedulers.scheduling_ddim.DDIMScheduler,
 diffusers.schedulers.scheduling_heun_discrete.HeunDiscreteScheduler,
 diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
 diffusers.schedulers.scheduling_deis_multistep.DEISMultistepScheduler,
 diffusers.schedulers.scheduling_pndm.PNDMScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2AncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2DiscreteScheduler,
 diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
 diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
 diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
 diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
 diffusers.schedulers.scheduling_unipc_multistep.UniPCMultiste

In [11]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [12]:
positive_prompt = """
(high detailed skin:1.2, high quality, reportage, journalism),
8k uhd, dslr, soft lighting, film grain, Fujifilm XT3
"""

negative_prompt="""
((text:1.5, deformed iris, deformed pupils, deformed hands, nsfw)),
((semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4)),
close up, cropped, out of frame, worst quality, low quality, jpeg artifacts, ugly, duplicate,
morbid, mutilated, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation,
deformed, blurry, burns, dehydrated, bad anatomy, bad proportions, extra limbs, cloned face, disfigured,
gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs,
fused fingers, too many fingers, long neck
"""

In [16]:
## LLM To create thumbnail prompt from scenes
from langchain import LLMChain
from langchain.prompts.chat import (ChatPromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate, HumanMessagePromptTemplate)

template="""
You are a YouTube thumbnail artist, the human will provide you with a script for the episode.
Your task is to create a prompt that the human will use to generate the thumbnail.

The keywords you use in your prompt generation are very important, so choose them wisely.
The prompt must describe an enticing thumbnail with interesting colors matching the general mood of the human provided
script.

Format your response as a JSON array of objects as defined below, you will always generate 10 prompts.

an example element for a script about the Titanic :
{{
  "prompt": "The Titanic ship, in the bottom of the ocean, with sea stuff growing on it, Narrator's face in scuba gear doing thumbs up overlayed on the real image"
}}
"""
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template("{text}")

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [17]:
chain = LLMChain(llm=llms.RevGPTLLM(model="gpt-4"), prompt=chat_prompt)
prompts_raw = chain.run(open(utils.SCRIPT).read())
prompts_raw

'[\n  {\n    "prompt": "A young Magnus Carlsen intensely focused on a chessboard, with the words \'The Prodigy\' above him in bold letters. Include a sepia filter for a nostalgic mood."\n  },\n  {\n    "prompt": "A collage of Magnus Carlsen as a child solving puzzles, playing with Lego, and learning chess from his father. Overlay the images with a vibrant, energetic color scheme."\n  },\n  {\n    "prompt": "Magnus Carlsen shaking hands with Grandmaster Simen Agdestein, with a rocket taking off in the background symbolizing his skyrocketing chess rating. Use dynamic colors and lines for a sense of motion."\n  },\n  {\n    "prompt": "Magnus Carlsen sitting at a chessboard, with ghostly chess pieces floating around him to represent different openings. Use a mix of mysterious blues and vibrant oranges."\n  },\n  {\n    "prompt": "Magnus Carlsen holding a World Chess Champion trophy, with a montage of opponents like Anand, Karjakin, and Caruana in the background. Use a golden glow to signif

In [22]:
guidance_scale        = 0.7
num_inference_steps   = 30
num_images_per_prompt = 2
image_height = 512
image_width = 768

for i,line in enumerate(json.loads(prompts_raw)):
    prompt = line
    images = pipeline(
        prompt="{} {}".format(prompt, positive_prompt),
        negative_prompt=negative_prompt,
        height=image_height,
        width=image_width,
        num_images_per_prompt=num_images_per_prompt,
        num_inference_steps=num_inference_steps,
        #guidance_scale=guidance_scale,
    ).images
        
    for j,image in enumerate(images):
        image.save("{}/thumbnails/thumbnail_{}_{}.jpg".format(utils.PATH_PREFIX, i+1, j+1))

  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (85 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['lighting, film grain, fujifilm xt 3']


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

...

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['3']


  0%|          | 0/30 [00:00<?, ?it/s]

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['3']


  0%|          | 0/30 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

In [12]:
# Free GPU memory for upscaler
del pipeline
del images
torch.cuda.empty_cache()

In [1]:
from PIL import Image
from diffusers import StableDiffusionUpscalePipeline

import torch

pipeline = StableDiffusionUpscalePipeline.from_pretrained(
    "stabilityai/stable-diffusion-x4-upscaler",
    revision="fp16",
    torch_dtype=torch.float16
)

text_encoder/model.safetensors not found


In [2]:
from xformers.ops import MemoryEfficientAttentionFlashAttentionOp

pipeline = pipeline.to("cuda")
pipeline.enable_xformers_memory_efficient_attention(attention_op=MemoryEfficientAttentionFlashAttentionOp)
# Workaround for not accepting attention shape using VAE for Flash Attention
pipeline.vae.enable_xformers_memory_efficient_attention(attention_op=None)

In [3]:
pipeline.scheduler.compatibles

[diffusers.schedulers.scheduling_k_dpm_2_discrete.KDPM2DiscreteScheduler,
 diffusers.schedulers.scheduling_dpmsolver_multistep.DPMSolverMultistepScheduler,
 diffusers.schedulers.scheduling_ddim.DDIMScheduler,
 diffusers.schedulers.scheduling_lms_discrete.LMSDiscreteScheduler,
 diffusers.schedulers.scheduling_ddpm.DDPMScheduler,
 diffusers.schedulers.scheduling_pndm.PNDMScheduler,
 diffusers.schedulers.scheduling_dpmsolver_singlestep.DPMSolverSinglestepScheduler,
 diffusers.schedulers.scheduling_deis_multistep.DEISMultistepScheduler,
 diffusers.schedulers.scheduling_unipc_multistep.UniPCMultistepScheduler,
 diffusers.utils.dummy_torch_and_torchsde_objects.DPMSolverSDEScheduler,
 diffusers.schedulers.scheduling_euler_discrete.EulerDiscreteScheduler,
 diffusers.schedulers.scheduling_k_dpm_2_ancestral_discrete.KDPM2AncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_euler_ancestral_discrete.EulerAncestralDiscreteScheduler,
 diffusers.schedulers.scheduling_heun_discrete.HeunDiscret

In [4]:
from diffusers import DPMSolverMultistepScheduler

pipeline.scheduler = DPMSolverMultistepScheduler.from_config(pipeline.scheduler.config)

In [12]:
guidance_scale        = 0.7
num_inference_steps   = 20
num_images_per_prompt = 1
n = 5

for i,prompt in enumerate(scene_lines):
    for j in range(1,n+1):
        low_res_image = Image.open("outputs/robertson/scene_{}_{}.png".format(i+1,j)).convert("RGB")
        low_res_image = low_res_image.resize((256,256))

        image = pipeline(
            prompt="{} {}".format(prompt, positive_prompt),
            negative_prompt=negative_prompt,
            image=low_res_image,
            num_images_per_prompt=num_images_per_prompt,
            num_inference_steps=num_inference_steps,
            #guidance_scale=guidance_scale,    
        ).images[0]

        image.save("outputs/robertson/scene_{}_{}_upscaled.png".format(i+1,j))

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]